In [3]:
from data_preparation import get_metadata, get_series,clip_missing_prone_values, prepare_division_and_country_series

series_with_heavy_missing, metadata_with_heavy_missing = get_series(), get_metadata()
division_missing_counts, metadata, series = clip_missing_prone_values(metadata_with_heavy_missing, series_with_heavy_missing)
region_series, metadata_region, country_series, metadata_country = prepare_division_and_country_series(series,
                                                                                                       metadata)

In [ ]:
# from visualization import box_plot_series
from data_preparation import get_category_info
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
colorScale, categoryName, AQScale = get_category_info()